### **Section 1: Environment Setup and Dependency Management**

* Core Libraries: We use torch for GPU-accelerated tensor operations and the diffusers library from Hugging Face to manage the Stable Diffusion XL (SDXL) pipelines.

* Interface and Processing: gradio is used to build the interactive laboratory environment, while PIL handles image encoding/decoding.

* Optimization: The cell includes gc (garbage collection) to manually clear the system RAM, which is vital for maintaining stability on a T4 GPU with limited memory.

In [9]:
import gradio as gr
import torch
import gc
import numpy as np
import uuid
import os
from dataclasses import dataclass
from diffusers import StableDiffusionXLPipeline, AutoPipelineForImage2Image, EulerDiscreteScheduler, AutoencoderKL
from PIL import Image
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')


### **Section 2: Data Infrastructure (The "Specimen" and Database)**
Rather than treating images as static files, this cell establishes a custom data structure that treats every generation as a biological entity.

* The Specimen Class: This object stores the "genetic material" of an image. The most important field is the dna, a 128x128 latent noise vector (tensor) that defines the structural layout of the image. It also tracks the lineage of the specimen, recording its generation number and parent IDs.

* **Database**: It maintains the current_generation for active breeding and a full_history to ensure that users can revisit and breed ancestors from any point in the project’s timeline.


In [10]:
@dataclass
class Specimen:
    id: str
    filepath: str
    dna: torch.Tensor
    genetic_prompt: str
    current_prompt: str
    generation: int
    parent_id: str
    quality_score: float = 0.0
    seed: int = None
    timestamp: str = None

class LaboratoryDatabase:
    def __init__(self):
        self.current_generation = []
        self.full_history = []
        self.temp_dir = "/tmp/darwin_studio"
        os.makedirs(self.temp_dir, exist_ok=True)

    def save_image(self, image, quality=95):
        filename = f"{uuid.uuid4()}.png"
        path = os.path.join(self.temp_dir, filename)
        image.save(path, format='PNG', optimize=True, quality=quality)
        return path

    def add(self, specimen):
        specimen.timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.current_generation.append(specimen)
        self.full_history.append(specimen)

    def get_current(self, index):
        if 0 <= index < len(self.current_generation):
            return self.current_generation[index]
        return None

    def get_from_history(self, index):
        if 0 <= index < len(self.full_history):
            return self.full_history[index]
        return None

    def clear_current(self):
        self.current_generation = []

    def get_current_gallery(self):
        items = []
        for i, s in enumerate(self.current_generation):
            quality_bar = "█" * int(s.quality_score * 10) if s.quality_score > 0 else ""
            label = f"Gen {s.generation} • Q:{s.quality_score:.2f} {quality_bar}\n{s.genetic_prompt[:40]}..."
            items.append((s.filepath, label))
        return items

    def get_history_gallery(self):
        items = []
        for i, s in enumerate(self.full_history):
            quality_bar = "█" * int(s.quality_score * 10) if s.quality_score > 0 else ""
            label = f"#{i+1} | Gen {s.generation} • Q:{s.quality_score:.2f}\n{s.genetic_prompt[:35]}..."
            items.append((s.filepath, label))
        return items

    def get_generations_grouped(self):
        generations = {}
        for s in self.full_history:
            if s.generation not in generations:
                generations[s.generation] = []
            generations[s.generation].append(s)
        return generations

    def get_stats(self):
        if not self.current_generation:
            return {
                "current": len(self.current_generation),
                "max_gen": 0,
                "avg_quality": 0,
                "best_quality": 0,
                "total": len(self.full_history)
            }

        total = len(self.current_generation)
        max_gen = max(s.generation for s in self.current_generation)
        avg_quality = sum(s.quality_score for s in self.current_generation) / total if total > 0 else 0
        best_quality = max(s.quality_score for s in self.current_generation) if total > 0 else 0

        return {
            "current": total,
            "max_gen": max_gen,
            "avg_quality": avg_quality,
            "best_quality": best_quality,
            "total": len(self.full_history)
        }


### **Section 3: Style Presets (Phenotype Mapping)**
To simplify the user experience, this cell defines the STYLE_PRESETS.

Purpose: While the DNA defines the structure, these presets define the aesthetic expression (e.g., "Photorealistic," "Cinematic," or "Oil Painting").


In [11]:

STYLE_PRESETS = {
    "None": {"suffix": "", "negative": ""},
    "Photorealistic": {
        "suffix": ", photorealistic, 8k uhd, high detail, sharp focus, professional photography",
        "negative": "ugly, blurry, cartoon, anime, painting, low quality, distorted"
    },
    "Cinematic": {
        "suffix": ", cinematic shot, dramatic lighting, film grain, movie still, professional color grading",
        "negative": "amateur, snapshot, blurry, overexposed, low quality"
    },
    "Oil Painting": {
        "suffix": ", oil painting, canvas texture, brushstrokes visible, classical art, masterpiece",
        "negative": "photograph, digital art, 3d render, blurry, low quality"
    },
    "Anime": {
        "suffix": ", anime style, cel shaded, vibrant colors, manga art, detailed illustration",
        "negative": "realistic, photograph, western cartoon, blurry, low quality"
    },
    "Cyberpunk": {
        "suffix": ", cyberpunk aesthetic, neon lights, futuristic, dystopian, high contrast, detailed",
        "negative": "natural, pastoral, vintage, historical, blurry"
    },
    "Fantasy Art": {
        "suffix": ", fantasy art, magical atmosphere, ethereal lighting, concept art, highly detailed",
        "negative": "modern, contemporary, realistic photo, blurry"
    },
    "Studio Portrait": {
        "suffix": ", professional studio portrait, soft lighting, shallow depth of field, 85mm lens",
        "negative": "full body, landscape, blurry, distorted face, multiple people"
    },
    "Epic Landscape": {
        "suffix": ", epic landscape photography, golden hour, dramatic sky, wide angle, 8k",
        "negative": "people, portrait, closeup, blurry, low quality"
    },
}


### **Section 4: The Darwin Engine (The Computational Brain)**
This cell loads the heavy-duty models and configures the memory optimization settings required to run SDXL on consumer-grade hardware.

**Model Choice:** We use `RealVisXL_V4.0_Lightning`. This is a "distilled" model, meaning it can produce high-quality photorealistic images in just 4 to 8 steps, rather than the standard 30 to 50.

**The VAE Fix:** We utilize the `madebyollin/sdxl-vae-fp16-fix`. Standard SDXL models often crash or produce black images when running in half-precision (FP16). This specialized VAE is fine-tuned to handle FP16 safely, saving massive amounts of VRAM.

**Memory Optimizations:**

* VAE Slicing: Instead of decoding a large image all at once, the engine processes it in sequential slices to prevent "Out of Memory" (OOM) errors.

* Attention Slicing: This breaks down the heavy internal calculations of the UNet, trading a small amount of speed for massive stability.

In [12]:
class DarwinEngine:
    def __init__(self):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"🚀 Initializing Darwin Studio on {self.device}...")

        if not torch.cuda.is_available():
            raise RuntimeError("CUDA not available! This app requires GPU.")

        self.cleanup()

        vae = AutoencoderKL.from_pretrained(
            "madebyollin/sdxl-vae-fp16-fix",
            torch_dtype=torch.float16
        )

        self.txt2img = StableDiffusionXLPipeline.from_pretrained(
            "SG161222/RealVisXL_V4.0_Lightning",
            vae=vae,
            torch_dtype=torch.float16,
            variant="fp16",
            use_safetensors=True
        )

        self.img2img = AutoPipelineForImage2Image.from_pipe(self.txt2img)

        conf = self.txt2img.scheduler.config
        for pipe in [self.txt2img, self.img2img]:
            pipe.scheduler = EulerDiscreteScheduler.from_config(
                conf, timestep_spacing="trailing", use_karras_sigmas=True
            )

        print("⚡ Moving models to GPU...")
        for pipe in [self.txt2img, self.img2img]:
            pipe.to(self.device)
            pipe.vae.enable_slicing()
            pipe.set_progress_bar_config(disable=True)

        try:
            for pipe in [self.txt2img, self.img2img]:
                pipe.enable_xformers_memory_efficient_attention()
            print("✅ xFormers enabled")
        except:
            try:
                for pipe in [self.txt2img, self.img2img]:
                    pipe.enable_attention_slicing(1)
                print("✅ Attention slicing enabled")
            except:
                pass

        print(f"✅ Engine Ready")

    def cleanup(self):
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.synchronize()

    def generate_dna(self, batch_size=1, seed=None):
        if seed is not None:
            generator = torch.Generator(device=self.device).manual_seed(seed)
            return torch.randn(
                (batch_size, 4, 128, 128),
                device=self.device,
                dtype=torch.float16,
                generator=generator
            )
        else:
            return torch.randn(
                (batch_size, 4, 128, 128),
                device=self.device,
                dtype=torch.float16
            )

    def image_to_dna(self, image):
        if isinstance(image, str):
            image = Image.open(image).convert("RGB")
        else:
            image = image.convert("RGB")

        image = image.resize((1024, 1024), Image.LANCZOS)

        img_array = np.array(image).astype(np.float32) / 255.0
        img_tensor = torch.from_numpy(img_array).permute(2, 0, 1).unsqueeze(0)
        img_tensor = (img_tensor * 2.0) - 1.0
        img_tensor = img_tensor.to(self.device, dtype=torch.float16)

        with torch.inference_mode():
            latents = self.txt2img.vae.encode(img_tensor).latent_dist.sample()
            latents = latents * self.txt2img.vae.config.scaling_factor

        return latents

    def calculate_quality_score(self, image):
        try:
            img_array = np.array(image.convert('L'))
            laplacian = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]])
            from scipy.ndimage import convolve
            sharpness = convolve(img_array.astype(float), laplacian).var()
            quality = min(sharpness / 1000.0, 10.0) / 10.0
            return quality
        except:
            return 0.0

    def render_txt2img(self, dna, prompt, neg_prompt, steps=8, guidance=2.0):
        if dna.device != torch.device(self.device):
            dna = dna.to(self.device)

        if steps not in [4, 6, 8, 10]:
            steps = 8

        with torch.inference_mode():
            (
                prompt_embeds,
                negative_prompt_embeds,
                pooled_prompt_embeds,
                negative_pooled_prompt_embeds,
            ) = self.txt2img.encode_prompt(
                prompt=prompt,
                negative_prompt=neg_prompt,
                device=self.device,
            )

            self.txt2img.scheduler.set_timesteps(steps, device=self.device)
            timesteps = self.txt2img.scheduler.timesteps

            latents = dna * self.txt2img.scheduler.init_noise_sigma

            for i, t in enumerate(timesteps):
                latent_model_input = torch.cat([latents] * 2) if guidance > 1.0 else latents
                latent_model_input = self.txt2img.scheduler.scale_model_input(latent_model_input, t)

                noise_pred = self.txt2img.unet(
                    latent_model_input,
                    t,
                    encoder_hidden_states=torch.cat([negative_prompt_embeds, prompt_embeds]) if guidance > 1.0 else prompt_embeds,
                    added_cond_kwargs={
                        "text_embeds": torch.cat([negative_pooled_prompt_embeds, pooled_prompt_embeds]) if guidance > 1.0 else pooled_prompt_embeds,
                        "time_ids": self.txt2img._get_add_time_ids(
                            (1024, 1024), (0, 0), (1024, 1024),
                            dtype=prompt_embeds.dtype,
                            text_encoder_projection_dim=1280
                        ).to(self.device).repeat(2 if guidance > 1.0 else 1, 1),
                    },
                ).sample

                if guidance > 1.0:
                    noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
                    noise_pred = noise_pred_uncond + guidance * (noise_pred_text - noise_pred_uncond)

                latents = self.txt2img.scheduler.step(noise_pred, t, latents).prev_sample

            latents = latents / self.txt2img.vae.config.scaling_factor
            image = self.txt2img.vae.decode(latents).sample
            image = self.txt2img.image_processor.postprocess(image, output_type="pil")[0]

        return image

    def render_img2img(self, image_input, prompt, neg_prompt, strength, steps=8):
        if isinstance(image_input, str):
            source_img = Image.open(image_input).convert("RGB")
        elif isinstance(image_input, Image.Image):
            source_img = image_input.convert("RGB")
        else:
            raise ValueError(f"Unexpected image_input type: {type(image_input)}")

        source_img = source_img.resize((1024, 1024), Image.LANCZOS)

        with torch.inference_mode():
            img = self.img2img(
                prompt=prompt,
                negative_prompt=neg_prompt,
                image=source_img,
                strength=strength,
                num_inference_steps=steps,
                guidance_scale=2.5,
            ).images[0]

        return img


### **Section 5: Evolutionary Logic**

This cell contains the mathematical formulas that drive the biological simulation.

1. Mutation (Evolution): This function injects Gaussian
noise into a parent’s DNA. By adjusting the "mutation rate," users can control how much the child differs from the parent. After mixing, we use renormalization—dividing the tensor by its own standard deviation—to ensure the resulting image stays sharp and doesn't become a "gray goo" of pixels.

2. Breeding (Hybridization): When two parents are bred, we use SLERP (Spherical Linear Interpolation). Standard averages (Linear Interpolation) often produce blurry images because they don't follow the "curve" of the AI's internal logic. SLERP follows the arc of the hypersphere, ensuring the hybrid child is as sharp as both parents.


In [13]:
def mutate_dna(parent_dna, mutation_rate, device, seed=None):
    """Controlled mutation"""
    if len(parent_dna.shape) == 3:
        parent_dna = parent_dna.unsqueeze(0)

    if parent_dna.device != torch.device(device):
        parent_dna = parent_dna.to(device)

    if seed is not None:
        generator = torch.Generator(device=device).manual_seed(seed)
        noise = torch.randn(
            parent_dna.shape,
            device=device,
            dtype=torch.float16,
            generator=generator
        )
    else:
        noise = torch.randn_like(parent_dna, device=device, dtype=torch.float16)

    child = (parent_dna * (1.0 - mutation_rate)) + (noise * mutation_rate)

    # Preserve variance
    parent_std = parent_dna.std()
    child_std = child.std()
    if child_std > 0:
        child = child * (parent_std / child_std)

    return child

def breed_dna_with_traits(parent1_dna, parent2_dna, parent1_weight, device, seed=None):
    if parent1_dna.device != torch.device(device):
        parent1_dna = parent1_dna.to(device)
    if parent2_dna.device != torch.device(device):
        parent2_dna = parent2_dna.to(device)

    if len(parent1_dna.shape) == 3:
        parent1_dna = parent1_dna.unsqueeze(0)
    if len(parent2_dna.shape) == 3:
        parent2_dna = parent2_dna.unsqueeze(0)

    parent2_weight = 1.0 - parent1_weight

    p1_norm = torch.nn.functional.normalize(parent1_dna.flatten(), dim=0)
    p2_norm = torch.nn.functional.normalize(parent2_dna.flatten(), dim=0)

    dot_product = (p1_norm * p2_norm).sum()
    dot_product = torch.clamp(dot_product, -1.0, 1.0)
    omega = torch.acos(dot_product)

    so = torch.sin(omega)
    if so.abs() < 1e-6:
        child_flat = parent1_weight * parent1_dna.flatten() + parent2_weight * parent2_dna.flatten()
    else:
        child_flat = (torch.sin((1.0 - parent1_weight) * omega) / so) * parent1_dna.flatten() + \
                     (torch.sin(parent1_weight * omega) / so) * parent2_dna.flatten()

    child = child_flat.reshape(parent1_dna.shape)

    p1_mean = parent1_dna.mean()
    p1_std = parent1_dna.std()
    p2_mean = parent2_dna.mean()
    p2_std = parent2_dna.std()

    target_mean = parent1_weight * p1_mean + parent2_weight * p2_mean
    target_std = parent1_weight * p1_std + parent2_weight * p2_std

    child_mean = child.mean()
    child_std = child.std()

    if child_std > 1e-6:
        child = (child - child_mean) / child_std
        child = child * target_std + target_mean

    if seed is not None:
        generator = torch.Generator(device=device).manual_seed(seed)
        noise = torch.randn(
            child.shape,
            device=device,
            dtype=torch.float16,
            generator=generator
        ) * 0.02 * target_std
    else:
        noise = torch.randn_like(child, device=device, dtype=torch.float16) * 0.02 * target_std

    child = child + noise

    final_std = child.std()
    if final_std > 1e-6:
        child = child * (target_std / final_std)

    return child


In [14]:

engine = DarwinEngine()
db = LaboratoryDatabase()

def handle_init(prompt, neg_prompt, seed, style_preset, batch_size, quality_mode, progress=gr.Progress()):
    """Initialize population"""
    try:
        db.clear_current()

        genetic_blueprint = prompt
        full_prompt = prompt
        full_neg = neg_prompt

        if style_preset != "None":
            full_prompt = prompt + STYLE_PRESETS[style_preset]["suffix"]
            full_neg = neg_prompt + ", " + STYLE_PRESETS[style_preset]["negative"]

        if quality_mode == "Ultra":
            full_prompt += ", masterpiece, best quality, sharp focus, highly detailed"
            full_neg += ", blurry, low quality, worst quality, soft focus"
            guidance = 3.0
        elif quality_mode == "High":
            full_prompt += ", high quality, detailed"
            full_neg += ", low quality, blurry"
            guidance = 2.5
        else:
            guidance = 2.0

        s_val = int(seed) if seed else None

        for i in range(batch_size):
            progress((i/batch_size), desc=f"🧬 Generating {i+1}/{batch_size}...")

            current_seed = (s_val + i) if s_val else None
            dna = engine.generate_dna(batch_size=1, seed=current_seed)

            img = engine.render_txt2img(dna, full_prompt, full_neg, steps=8, guidance=guidance)
            path = db.save_image(img)
            quality = engine.calculate_quality_score(img)

            specimen = Specimen(
                id=str(uuid.uuid4()),
                filepath=path,
                dna=dna,
                genetic_prompt=genetic_blueprint,
                current_prompt=full_prompt,
                generation=1,
                parent_id="Genesis",
                quality_score=quality,
                seed=current_seed
            )
            db.add(specimen)

        engine.cleanup()

        timeline_data = get_timeline_gallery()

        stats = db.get_stats()
        return (
            timeline_data,
            f"✅ Genesis complete • Generated {batch_size} specimens",
            f"Total Specimens: **{stats['total']}** | Max Gen: **{stats['max_gen']}** | Best Quality: **{stats['best_quality']:.2f}**"
        )
    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()
        return [], f"❌ Error: {str(e)}", "Total: **0**"

def handle_evolve(selected_idx, mutation_rate, num_variants, quality_mode, progress=gr.Progress()):
    """Evolution from current generation"""
    if selected_idx is None:
        timeline = get_timeline_and_display()
        return timeline, "⚠️ Select a specimen", get_stats_text()

    parent = db.get_from_history(selected_idx)
    if not parent:
        timeline = get_timeline_and_display()
        return timeline, "⚠️ Invalid selection", get_stats_text()

    db.clear_current()

    genetic_blueprint = parent.genetic_prompt
    render_prompt = genetic_blueprint
    neg_prompt = "ugly, blurry, low quality"

    if quality_mode == "Ultra":
        render_prompt += ", masterpiece, best quality, sharp focus"
        neg_prompt += ", blurry, low quality, worst quality"
        guidance = 3.0
    elif quality_mode == "High":
        render_prompt += ", high quality, detailed"
        neg_prompt += ", low quality"
        guidance = 2.5
    else:
        guidance = 2.0

    try:
        for i in range(num_variants):
            progress((i/num_variants), desc=f"🧬 Evolving {i+1}/{num_variants}...")

            mutation_seed = hash(parent.id + str(i)) % (2**32)
            child_dna = mutate_dna(parent.dna, mutation_rate, engine.device, seed=mutation_seed)

            img = engine.render_txt2img(child_dna, render_prompt, neg_prompt, steps=8, guidance=guidance)
            path = db.save_image(img)
            quality = engine.calculate_quality_score(img)

            child = Specimen(
                id=str(uuid.uuid4()),
                filepath=path,
                dna=child_dna,
                genetic_prompt=genetic_blueprint,
                current_prompt=render_prompt,
                generation=parent.generation + 1,
                parent_id=parent.id,
                quality_score=quality,
                seed=mutation_seed
            )
            db.add(child)

        engine.cleanup()
        timeline = get_timeline_and_display()
        return (
            timeline,
            f"✅ Evolution complete • Generation {parent.generation + 1}",
            get_stats_text()
        )

    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()
        engine.cleanup()
        timeline = get_timeline_and_display()
        return timeline, f"❌ Error: {str(e)}", get_stats_text()

def handle_morph(selected_idx, new_prompt, strength, num_variants, quality_mode, progress=gr.Progress()):
    """Morph from current generation with variants"""
    if selected_idx is None:
        timeline = get_timeline_and_display()
        return timeline, "⚠️ Select a specimen", get_stats_text()

    if not new_prompt or new_prompt.strip() == "":
        timeline = get_timeline_and_display()
        return timeline, "⚠️ Enter morph prompt", get_stats_text()

    parent = db.get_from_history(selected_idx)
    if not parent:
        timeline = get_timeline_and_display()
        return timeline, "⚠️ Invalid selection", get_stats_text()

    db.clear_current()

    new_genetic_blueprint = new_prompt
    neg_prompt = "ugly, blurry, low quality"

    if quality_mode == "Ultra":
        neg_prompt += ", blurry, low quality, worst quality"
    elif quality_mode == "High":
        neg_prompt += ", low quality"

    try:
        strength_variations = []
        for i in range(num_variants):
            variation = strength + (i - num_variants/2) * 0.1
            variation = max(0.3, min(0.9, variation))
            strength_variations.append(variation)

        for i, variant_strength in enumerate(strength_variations):
            progress((i/num_variants), desc=f"🎨 Morphing {i+1}/{num_variants}...")

            img = engine.render_img2img(parent.filepath, new_prompt, neg_prompt, variant_strength, steps=8)
            path = db.save_image(img)
            new_dna = engine.image_to_dna(img)
            quality = engine.calculate_quality_score(img)

            child = Specimen(
                id=str(uuid.uuid4()),
                filepath=path,
                dna=new_dna,
                genetic_prompt=new_genetic_blueprint,
                current_prompt=new_prompt,
                generation=parent.generation + 1,
                parent_id=parent.id,
                quality_score=quality
            )
            db.add(child)

        engine.cleanup()
        timeline = get_timeline_and_display()
        return (
            timeline,
            f"✅ Morph complete • Generation {parent.generation + 1}",
            get_stats_text()
        )

    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()
        engine.cleanup()
        timeline = get_timeline_and_display()
        return timeline, f"❌ Error: {str(e)}", get_stats_text()

def handle_breed(parent1_idx, parent2_idx, trait1, trait2, blend_ratio, num_variants, quality_mode, progress=gr.Progress()):
    if parent1_idx is None or parent2_idx is None:
        timeline = get_timeline_and_display()
        return timeline, "⚠️ Select two parents", get_stats_text()

    if parent1_idx == parent2_idx:
        timeline = get_timeline_and_display()
        return timeline, "⚠️ Select DIFFERENT parents", get_stats_text()

    parent1 = db.get_from_history(parent1_idx)
    parent2 = db.get_from_history(parent2_idx)

    if not parent1 or not parent2:
        timeline = get_timeline_and_display()
        return timeline, "⚠️ Invalid selection", get_stats_text()

    trait1_text = trait1.strip() if trait1 and trait1.strip() else parent1.genetic_prompt
    trait2_text = trait2.strip() if trait2 and trait2.strip() else parent2.genetic_prompt

    hybrid_genetic_prompt = f"{trait1_text}, {trait2_text}"

    neg_prompt = "ugly, blurry, low quality, distorted, deformed"
    if quality_mode == "Ultra":
        hybrid_genetic_prompt += ", masterpiece, best quality, sharp focus, highly detailed"
        neg_prompt += ", blurry, soft focus, low quality, worst quality"
        guidance = 3.5
    elif quality_mode == "High":
        hybrid_genetic_prompt += ", high quality, detailed"
        neg_prompt += ", low quality"
        guidance = 3.0
    else:
        guidance = 2.5

    db.clear_current()

    try:
        blend_variations = []
        for i in range(num_variants):
            variation = blend_ratio + (i - num_variants/2) * 0.15
            variation = max(0.2, min(0.8, variation))
            blend_variations.append(variation)

        for i, ratio in enumerate(blend_variations):
            progress((i/num_variants), desc=f"🔬 Breeding offspring {i+1}/{num_variants}...")

            breed_seed = hash(f"{parent1.id}{parent2.id}{i}") % (2**32)

            child_dna = breed_dna_with_traits(
                parent1.dna,
                parent2.dna,
                ratio,
                engine.device,
                seed=breed_seed
            )

            img = engine.render_txt2img(
                child_dna,
                hybrid_genetic_prompt,
                neg_prompt,
                steps=8,
                guidance=guidance
            )

            path = db.save_image(img)
            quality = engine.calculate_quality_score(img)

            child = Specimen(
                id=str(uuid.uuid4()),
                filepath=path,
                dna=child_dna,
                genetic_prompt=hybrid_genetic_prompt,
                current_prompt=hybrid_genetic_prompt,
                generation=max(parent1.generation, parent2.generation) + 1,
                parent_id=f"{parent1.id[:8]}+{parent2.id[:8]}",
                quality_score=quality,
                seed=breed_seed
            )
            db.add(child)

        engine.cleanup()
        timeline = get_timeline_and_display()
        return (
            timeline,
            f"✅ Breeding complete • Gen {max(parent1.generation, parent2.generation) + 1}",
            get_stats_text()
        )

    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()
        engine.cleanup()
        timeline = get_timeline_and_display()
        return timeline, f"❌ Error: {str(e)}", get_stats_text()

def get_timeline_and_display():
    return get_timeline_gallery()

def get_timeline_gallery():
    generations = db.get_generations_grouped()
    timeline_items = []

    for gen_num in sorted(generations.keys()):
        for specimen in generations[gen_num]:
            label = f"#{len(timeline_items)+1} | Gen {gen_num}"
            timeline_items.append((specimen.filepath, label))

    return timeline_items

def get_stats_text():
    stats = db.get_stats()
    return f"Total Specimens: **{stats['total']}** | Max Gen: **{stats['max_gen']}** | Best Quality: **{stats['best_quality']:.2f}**"

def handle_show_specimen_details(evt: gr.SelectData):
    if evt is None or evt.index >= len(db.full_history):
        return "*Select a specimen to view details*"

    specimen = db.full_history[evt.index]
    quality_stars = "★" * int(specimen.quality_score * 5) if specimen.quality_score > 0 else "☆☆☆☆☆"

    details = f"""**SPECIMEN #{evt.index + 1}**

**Generation:** {specimen.generation}
**Parent ID:** {specimen.parent_id}
**Quality:** {quality_stars} ({specimen.quality_score:.3f})
**Seed:** {specimen.seed if specimen.seed else "Random"}
**Created:** {specimen.timestamp}

**Genetic Blueprint:**
_{specimen.genetic_prompt}_
"""
    return details

def handle_select_for_breed(evt: gr.SelectData, parent1, parent2, mode):
    if evt is None:
        return parent1, parent2, "⚠️ Error selecting specimen"

    # Only process if we're in breeding mode
    if mode not in ["A", "B"]:
        return parent1, parent2, "*Click 'SELECT PARENT A' or 'SELECT PARENT B' button first*"

    idx = evt.index

    if mode == "A":
        specimen = db.get_from_history(idx)
        if specimen:
            return idx, parent2, f"✅ Parent A selected: Specimen #{idx + 1} (Gen {specimen.generation})"
        return parent1, parent2, "⚠️ Invalid selection"
    else:  # mode == "B"
        if parent1 is None:
            return parent1, parent2, "⚠️ Please select Parent A first"
        specimen = db.get_from_history(idx)
        if specimen:
            return parent1, idx, f"✅ Ready to breed! Parent A: #{parent1 + 1} × Parent B: #{idx + 1}"
        return parent1, parent2, "⚠️ Invalid selection"


🚀 Initializing Darwin Studio on cuda...


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

⚡ Moving models to GPU...
✅ Attention slicing enabled
✅ Engine Ready


### **Section 5: The Laboratory Interface (Gradio UI)**
The final cell designs the web interface, providing a visual environment for selective breeding.


In [15]:

# ==========================================
# 6. IMPROVED UI WITH BETTER UX
# ==========================================
css = """
:root {
    --bg: #0a0a0f;
    --panel: #13131f;
    --accent: #00f2ff;
    --accent-dim: #00f2ff33;
    --text: #e0e0e0;
    --text-dim: #888;
    --border: #2a2a35;
    --success: #00ff9d;
}

.gradio-container {
    background: var(--bg) !important;
    font-family: 'Segoe UI', Roboto, sans-serif !important;
}

.sidebar {
    background: var(--panel) !important;
    border-right: 1px solid var(--border) !important;
    padding: 20px !important;
}

.logo {
    font-size: 24px !important;
    font-weight: 700 !important;
    color: var(--accent) !important;
    text-align: center !important;
    margin-bottom: 20px !important;
}

.control-group {
    background: #00000033 !important;
    padding: 15px !important;
    border-radius: 8px !important;
    border: 1px solid var(--border) !important;
    margin-bottom: 15px !important;
}

button {
    width: 100% !important;
    background: var(--accent) !important;
    color: #000 !important;
    border: none !important;
    padding: 12px !important;
    font-weight: 600 !important;
    cursor: pointer !important;
    border-radius: 4px !important;
    text-transform: uppercase !important;
}

button:hover {
    background: #fff !important;
    box-shadow: 0 0 15px var(--accent) !important;
}

/* Breeding mode cursor indication */
.breeding-mode-a {
    cursor: crosshair !important;
}

.breeding-mode-b {
    cursor: crosshair !important;
}

.timeline-container {
    background: var(--panel) !important;
    padding: 15px !important;
    border-radius: 8px !important;
    border: 1px solid var(--border) !important;
    margin-bottom: 20px !important;
}

.main-display {
    background: var(--bg) !important;
    padding: 20px !important;
}

.status-box {
    background: var(--panel) !important;
    border-left: 4px solid var(--accent) !important;
    padding: 15px !important;
    border-radius: 4px !important;
    margin: 10px 0 !important;
}

/* IMPROVED GALLERY LABELS - Better contrast and visibility */
.gallery-item {
    position: relative !important;
}

.gallery-item .caption-label {
    background: rgba(0, 0, 0, 0.85) !important;
    color: #ffffff !important;
    padding: 8px 12px !important;
    font-size: 13px !important;
    font-weight: 600 !important;
    border-radius: 4px !important;
    text-shadow: 1px 1px 2px rgba(0, 0, 0, 0.8) !important;
}

.gallery img {
    border: 1px solid var(--border) !important;
    border-radius: 8px !important;
    transition: 0.3s !important;
}

.gallery img:hover {
    border-color: var(--accent) !important;
    box-shadow: 0 0 20px var(--accent) !important;
}

/* Make text visible in gallery captions */
div[data-testid="block-label"] {
    color: #ffffff !important;
}

.gr-gallery .caption {
    background: rgba(0, 0, 0, 0.9) !important;
    color: #ffffff !important;
    font-weight: 600 !important;
    padding: 6px 10px !important;
}
"""

with gr.Blocks(title="Darwin Studio", theme=gr.themes.Base(), css=css) as app:

    # States
    breed_parent1_state = gr.State(None)
    breed_parent2_state = gr.State(None)
    breed_mode_state = gr.State(None)  # None, "A", or "B"
    current_selected_state = gr.State(None)

    with gr.Row():
        # LEFT SIDEBAR
        with gr.Column(scale=2, min_width=320, elem_classes="sidebar"):
            gr.HTML('<div class="logo">🧬 DARWIN STUDIO</div>')

            # GENESIS
            with gr.Group(elem_classes="control-group"):
                gr.Markdown("### 🌟 Genesis")
                master_prompt = gr.Textbox(
                    label="Prompt",
                    value="A futuristic robot in a neon-lit city",
                    lines=3
                )
                style_dropdown = gr.Dropdown(
                    choices=list(STYLE_PRESETS.keys()),
                    value="Cinematic",
                    label="Style"
                )
                with gr.Row():
                    batch_size = gr.Number(label="Batch", value=2, minimum=2, maximum=4, precision=0)
                    quality_mode = gr.Dropdown(choices=["Normal", "High", "Ultra"], value="High", label="Quality")
                btn_init = gr.Button("🎬 GENERATE")

            # EVOLVE
            with gr.Accordion("🧬 Evolve", open=False):
                with gr.Group(elem_classes="control-group"):
                    mutation_slider = gr.Slider(0.1, 0.9, value=0.35, step=0.05, label="Mutation")
                    evolve_variants = gr.Number(label="Variants", value=3, minimum=1, maximum=6, precision=0)
                    btn_evolve = gr.Button("EVOLVE")

            # MORPH
            with gr.Accordion("🎨 Morph", open=False):
                with gr.Group(elem_classes="control-group"):
                    morph_prompt = gr.Textbox(label="New Concept", lines=3)
                    strength_slider = gr.Slider(0.1, 0.9, value=0.65, step=0.05, label="Strength")
                    morph_variants = gr.Number(label="Variants", value=3, minimum=1, maximum=6, precision=0)
                    btn_morph = gr.Button("MORPH")

            # BREED
            with gr.Accordion("🔬 Breed", open=False):
                with gr.Group(elem_classes="control-group"):
                    gr.Markdown("**Step 1:** Click button, then click a specimen in timeline")
                    btn_select_a = gr.Button("📌 SELECT PARENT A", variant="secondary")

                    gr.Markdown("**Step 2:** Click button, then click another specimen")
                    btn_select_b = gr.Button("📌 SELECT PARENT B", variant="secondary")

                    breed_status = gr.Markdown("*No parents selected*")

                    breed_trait1 = gr.Textbox(label="Traits from Parent A (optional)", lines=2)
                    breed_trait2 = gr.Textbox(label="Traits from Parent B (optional)", lines=2)
                    blend_slider = gr.Slider(0.3, 0.7, value=0.5, step=0.05, label="DNA Balance (A ← → B)")
                    breed_variants = gr.Number(label="Variants", value=4, minimum=2, maximum=8, precision=0)
                    btn_breed = gr.Button("🔬 BREED")

            status_box = gr.Markdown("**Status:** Ready", elem_classes="status-box")

        # RIGHT MAIN AREA
        with gr.Column(scale=5, elem_classes="main-display"):
            stats_display = gr.Markdown("Total: **0** | Max Gen: **0** | Best: **0.00**")

            gr.Markdown("### 📊 Evolution Timeline")
            gr.Markdown("*All generations - Click any specimen to view details*")

            # Main timeline gallery
            timeline_gallery = gr.Gallery(
                label="Timeline",
                columns=6,
                rows=3,
                height=600,
                object_fit="cover",
                show_label=False,
                allow_preview=True,
                preview=True
            )

            # Details panel
            specimen_details = gr.Markdown("*Click a specimen to view details*", elem_classes="status-box")

    # EVENT HANDLERS

    # Genesis
    btn_init.click(
        handle_init,
        [master_prompt, gr.State("ugly, blurry, low quality"), gr.State(None), style_dropdown, batch_size, quality_mode],
        [timeline_gallery, status_box, stats_display]
    )

    # Timeline selection shows details
    timeline_gallery.select(
        handle_show_specimen_details,
        None,
        specimen_details
    )

    selected_specimen_state = gr.State(None)

    def on_timeline_click_for_operation(evt: gr.SelectData, current_mode):
        """Store selected specimen index for operations - only if NOT in breeding mode"""
        if evt is None:
            return None
        if current_mode in ["A", "B"]:
            return None
        return evt.index

    timeline_gallery.select(
        on_timeline_click_for_operation,
        breed_mode_state,
        selected_specimen_state
    )

    # Evolve
    btn_evolve.click(
        handle_evolve,
        [selected_specimen_state, mutation_slider, evolve_variants, quality_mode],
        [timeline_gallery, status_box, stats_display]
    )

    # Morph
    btn_morph.click(
        handle_morph,
        [selected_specimen_state, morph_prompt, strength_slider, morph_variants, quality_mode],
        [timeline_gallery, status_box, stats_display]
    )

    def activate_parent_a_mode():
        return "A", "*🎯 BREEDING MODE: Click a specimen in the timeline to select Parent A*"

    def activate_parent_b_mode():
        return "B", "*🎯 BREEDING MODE: Click a specimen in the timeline to select Parent B*"

    btn_select_a.click(
        activate_parent_a_mode,
        None,
        [breed_mode_state, breed_status]
    )

    btn_select_b.click(
        activate_parent_b_mode,
        None,
        [breed_mode_state, breed_status]
    )

    timeline_gallery.select(
        handle_select_for_breed,
        [breed_parent1_state, breed_parent2_state, breed_mode_state],
        [breed_parent1_state, breed_parent2_state, breed_status]
    ).then(
        lambda: None,
        None,
        breed_mode_state
    )

    btn_breed.click(
        handle_breed,
        [breed_parent1_state, breed_parent2_state, breed_trait1, breed_trait2, blend_slider, breed_variants, quality_mode],
        [timeline_gallery, status_box, stats_display]
    )


In [ ]:

if __name__ == "__main__":
    app.queue(max_size=20)
    app.launch(share=True, debug=True, server_port=7870)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://68bb83df16441d6012.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
